# Project Concrete 

_Project Concrete_ aims to correlate, visualize and predict real estate prices based on the relationship between various
factors such as unemployment rate, disposable income and population growth in Austria.

Data Sources used for this endeavour
- [Unemployment Rates (Source: data.gv.at)](https://www.data.gv.at/katalog/dataset/CFE2FF7E9AD53C1EE053C630070AB105)
- [Net Income Vienna (Source: statistik.at)](https://www.data.gv.at/katalog/dataset/d76c0e8b-c599-4700-8a88-29d0d87e563d)
- [Population Growth (Source: data.gv.at)](https://www.data.gv.at/katalog/dataset/f5f823c1-631b-35bd-abed-1442a7cb52a2)
- [Real Estate Prices Vienna (Source: data.gv.at)](https://www.data.gv.at/katalog/dataset/kaufpreissammlung-liegenschaften-wien/resource/7b9bdd2d-2ff0-4e6e-bba5-21483d8cf55b)
- ~~[Real Estate Prices (Source: statistik.at)](https://www.statistik.at/web_de/statistiken/wirtschaft/preise/immobilien_durchschnittspreise/index.html)~~
- ~~[Disposable Income  (Source: statistik.at)](https://www.statistik.at/web_de/statistiken/menschen_und_gesellschaft/soziales/haushalts-einkommen/index.html)~~

Planned execution steps:
- Data aggregation and collection (see Planned Data Sources)
- Data preperation and import into MongoDB
- Analysis of data via Jupyter Notebook and MapReduce
- Presentation and visualization of results (i.e. map of Austria/Vienna, etc.)


### Install pymongo

In [1]:
!pip install pymongo

### Connect to DB via connection string

In [3]:
import pymongo
from pymongo import MongoClient
import pandas as pd

# Provide the mongodb atlas url to connect python to mongodb using pymongo
CONNECTION_STRING = "mongodb://127.0.0.1:27017"

# Create a connection using MongoClient
myclient = MongoClient(CONNECTION_STRING)

print("Connection Successful")

# Check DB 
for db in myclient.list_databases():
    print(db)

Connection Successful
{'name': 'admin', 'sizeOnDisk': 40960, 'empty': False}
{'name': 'config', 'sizeOnDisk': 61440, 'empty': False}
{'name': 'local', 'sizeOnDisk': 40960, 'empty': False}


In [4]:
# Import Unemployment Data CSV from 1946
unemploy_df = pd.read_csv('ub_al_alq_os.csv', sep = ";", decimal=',')
unemploy_df

,Jahr,UnselbstBesch,ArbeitslosVorgemerkte,Arbeitslosenquote,OffeneStellen,Unnamed: 5
0,1946,1760000,74000,4.034896,140067,NaN
1,1947,1900000,52700,2.698827,102810,NaN
2,1948,1926700,54500,2.750858,45334,NaN
3,1949,1944700,99900,4.886041,35724,NaN
4,1950,1946886,128745,6.202692,25187,NaN
...,...,...,...,...,...,...
71,2017,3655297,339976,8.509456,56854,NaN
72,2018,3741484,312107,7.699519,71545,NaN
73,2019,3797304,301328,7.351916,77093,NaN
74,2020,3717164,409639,9.926304,62833,NaN


In [5]:
# Import Population Growth Data CSV from 1974
pop_growth_df = pd.read_csv('OGD_ake003j_AKEZR_1.csv', sep = ";", decimal=',')
pop_growth_df = pop_growth_df.rename(columns={'C-A10-0': 'YEAR', 'F-ISIS-1': 'PERSON_IN_THOUSAND'})
pop_growth_df.YEAR = pop_growth_df.YEAR.str.replace('A10-','',regex = True)
pop_growth_df

,YEAR,PERSON_IN_THOUSAND
0,1974,7519
1,1975,7501
2,1976,7484
3,1977,7485
4,1978,7482
5,1979,7467
6,1980,7465
7,1981,7481
8,1982,7493
9,1983,7479


In [19]:
# Import Real Estate Purchase list from 1990 (Kaufpreissammlung Liegenschaften Wien)
real_estate_vienna_df = pd.read_csv('kaufpreissammlung-liegenschaften.csv', sep = ";", decimal=',', encoding='latin-1', low_memory=False)
real_estate_vienna_df

,KG.Code,Katastralgemeinde,EZ,PLZ,Straße,ON,Gst.,Gst.Fl.,ErwArt,Erwerbsdatum,...,Baureifgest,% Widmung,Baurecht,Bis,auf EZ,Stammeinlage,sonst_wid,sonst_wid_prz,ber. Kaufpreis,Bauzins
0,1617,Strebersdorf,1417.0,1210.0,Mühlweg,13,752/16,755.0,Kaufvertrag,13.10.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,750000.0,NaN
1,1607,Groß Jedlersdorf II,193.0,1210.0,Bahnsteggasse,4,408,510.0,Kaufvertrag,13.09.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,1250000.0,NaN
2,1209,Ober St.Veit,3570.0,1130.0,Jennerplatz,34/20,938/3,456.0,Kaufvertrag,10.08.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,480000.0,NaN
3,1207,Lainz,405.0,1130.0,Sebastian-Brunner-Gasse,6,8/23,523.0,Kaufvertrag,30.12.2020,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,1600000.0,NaN
4,1101,Favoriten,3831.0,1100.0,Laxenburger Straße,2C -2 D,2044/19,12768.0,Kaufvertrag,04.11.2020,...,FALSCH,30.0,FALSCH,NaN,NaN,FALSCH,"W V 22 g , Wi g","40 ,30",15000000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57907,1204,Hadersdorf,1057.0,1140.0,Laskywiesengasse,10,889,1313.0,Kaufvertrag,18.08.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,950000.0,NaN
57908,1204,Hadersdorf,1200.0,1140.0,Robert-Fuchs-Gasse,25-31,448,4003.0,Kaufvertrag,05.12.2018,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,5200000.0,NaN
57909,1206,Hütteldorf,2760.0,1140.0,Ulmenstraße,48,1232/1,499.0,Kaufvertrag,15.06.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,699000.0,NaN
57910,1216,Weidlingau,5.0,1140.0,Hauptstraße,114,11/2,1649.0,Kaufvertrag,26.07.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,1496000.0,NaN


In [7]:
# Import Net Income Data in Vienna from 2002
net_income_vienna_df = pd.read_csv('vie-bez-biz-ecn-inc-sex-2002f.csv', sep=';', decimal=',')
net_income_vienna_df

,NUTS,DISTRICT_CODE,SUB_DISTRICT_CODE,REF_YEAR,REF_DATE,INC_TOT_VALUE,INC_MAL_VALUE,INC_FEM_VALUE
0,AT13,90000,90000,2002,20021231,18.217,20.709,15.424
1,AT13,90100,90100,2002,20021231,25.463,31.961,18.536
2,AT13,90200,90200,2002,20021231,16.439,18.301,14.282
3,AT13,90300,90300,2002,20021231,18.701,21.444,15.804
4,AT13,90400,90400,2002,20021231,20.325,23.641,16.876
...,...,...,...,...,...,...,...,...
451,AT13,91900,91900,2020,20201231,29.195,34.102,24.257
452,AT13,92000,92000,2020,20201231,20.671,21.591,19.531
453,AT13,92100,92100,2020,20201231,24.061,26.202,21.693
454,AT13,92200,92200,2020,20201231,26.272,29.302,23.100


### Create collection
Loading dataframes into database

In [8]:
# In MongoDB collections and DBs are only created on first insert (reminiscent of Git folders)
db = myclient["immodb"]

# We insert the whole dateframes into the DB

# Insert unemploymentData
db.unemploymentData.insert_many(unemploy_df.to_dict('records'))
# Insert populationGrowth
db.populationGrowth.insert_many(pop_growth_df.to_dict('records'))
# Insert realEstateVienna
db.realEstateVienna.insert_many(real_estate_vienna_df.to_dict('records'))
# Insert netIncomeVienna
db.netIncomeVienna.insert_many(net_income_vienna_df.to_dict('records'))

# Check if colletions have been created
print(db.list_collection_names())

['netIncomeVienna', 'populationGrowth', 'unemploymentData', 'realEstateVienna']


In [45]:
# Check if collections have data

data = [(unemp_col.find(), 'Unemployment Data'), (populationGrowth_col.find(), 'Population Growth'), (realEstateVienna_col.find(), 'Real Estate Sales'), (netIncomeVienna_col.find(), 'Net Income')]
for (cur, title) in data:
    print(title, '\nRow Count: ', len(list(cur)))

Unemployment Data 
Row Count:  76
Population Growth 
Row Count:  48
Real Estate Sales 
Row Count:  57912
Net Income 
Row Count:  456


In [115]:
# Drop all collections
# db["unemploymentData"].drop()
# db["populationGrowth"].drop()
# db["realEstateVienna"].drop()
# db["netIncomeVienna"].drop()

### Load from database
Build new dataframe by reading from database

In [48]:
# Unemployment Data 
unemployment_fromDb = pd.DataFrame(list(unemp_col.find()))
unemployment_fromDb = unemployment_fromDb.set_index("_id")
unemployment_fromDb

,Jahr,UnselbstBesch,ArbeitslosVorgemerkte,Arbeitslosenquote,OffeneStellen,Unnamed: 5
_id,,,,,,
6263f0aeeb832bd5a224a05d,1946,1760000,74000,4.034896,140067,NaN
6263f0aeeb832bd5a224a05e,1947,1900000,52700,2.698827,102810,NaN
6263f0aeeb832bd5a224a05f,1948,1926700,54500,2.750858,45334,NaN
6263f0aeeb832bd5a224a060,1949,1944700,99900,4.886041,35724,NaN
6263f0aeeb832bd5a224a061,1950,1946886,128745,6.202692,25187,NaN
...,...,...,...,...,...,...
6263f0aeeb832bd5a224a0a4,2017,3655297,339976,8.509456,56854,NaN
6263f0aeeb832bd5a224a0a5,2018,3741484,312107,7.699519,71545,NaN
6263f0aeeb832bd5a224a0a6,2019,3797304,301328,7.351916,77093,NaN


In [53]:
# Population Growth
populationGrowth_fromDb = pd.DataFrame(list(populationGrowth_col.find()))
populationGrowth_fromDb = populationGrowth_fromDb.set_index("_id")
populationGrowth_fromDb

,YEAR,PERSON_IN_THOUSAND
_id,,
6263f0aeeb832bd5a224a0a9,1974,7519
6263f0aeeb832bd5a224a0aa,1975,7501
6263f0aeeb832bd5a224a0ab,1976,7484
6263f0aeeb832bd5a224a0ac,1977,7485
6263f0aeeb832bd5a224a0ad,1978,7482
6263f0aeeb832bd5a224a0ae,1979,7467
6263f0aeeb832bd5a224a0af,1980,7465
6263f0aeeb832bd5a224a0b0,1981,7481
6263f0aeeb832bd5a224a0b1,1982,7493


In [13]:
# Real Estate Purchase List Vienna
realEstateVienna_fromDb = pd.DataFrame(list(realEstateVienna_col.find()))
realEstateVienna_fromDb = realEstateVienna_fromDb.set_index("_id")
realEstateVienna_fromDb

,KG.Code,Katastralgemeinde,EZ,PLZ,Straße,ON,Gst.,Gst.Fl.,ErwArt,Erwerbsdatum,...,Baureifgest,% Widmung,Baurecht,Bis,auf EZ,Stammeinlage,sonst_wid,sonst_wid_prz,ber. Kaufpreis,Bauzins
_id,,,,,,,,,,,,,,,,,,,,,
6263f0b5eb832bd5a224a0d9,1617,Strebersdorf,1417.0,1210.0,Mühlweg,13,752/16,755.0,Kaufvertrag,13.10.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,750000.0,NaN
6263f0b5eb832bd5a224a0da,1607,Groß Jedlersdorf II,193.0,1210.0,Bahnsteggasse,4,408,510.0,Kaufvertrag,13.09.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,1250000.0,NaN
6263f0b5eb832bd5a224a0db,1209,Ober St.Veit,3570.0,1130.0,Jennerplatz,34/20,938/3,456.0,Kaufvertrag,10.08.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,480000.0,NaN
6263f0b5eb832bd5a224a0dc,1207,Lainz,405.0,1130.0,Sebastian-Brunner-Gasse,6,8/23,523.0,Kaufvertrag,30.12.2020,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,1600000.0,NaN
6263f0b5eb832bd5a224a0dd,1101,Favoriten,3831.0,1100.0,Laxenburger Straße,2C -2 D,2044/19,12768.0,Kaufvertrag,04.11.2020,...,FALSCH,30.0,FALSCH,NaN,NaN,FALSCH,"W V 22 g , Wi g","40 ,30",15000000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263f0b6eb832bd5a225830c,1204,Hadersdorf,1057.0,1140.0,Laskywiesengasse,10,889,1313.0,Kaufvertrag,18.08.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,950000.0,NaN
6263f0b6eb832bd5a225830d,1204,Hadersdorf,1200.0,1140.0,Robert-Fuchs-Gasse,25-31,448,4003.0,Kaufvertrag,05.12.2018,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,5200000.0,NaN
6263f0b6eb832bd5a225830e,1206,Hütteldorf,2760.0,1140.0,Ulmenstraße,48,1232/1,499.0,Kaufvertrag,15.06.2021,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,699000.0,NaN


In [57]:
# Create Pandas Dataframe from Real Estate Purchase list  
netIncomeVienna_fromDb = pd.DataFrame(list(realEstateVienna_col.find()))
netIncomeVienna_fromDb = netIncomeVienna_fromDb.set_index("_id")

# Sort by date, time series
netIncomeVienna_fromDb.sort_values("Erwerbsdatum")

,KG.Code,Katastralgemeinde,EZ,PLZ,Straße,ON,Gst.,Gst.Fl.,ErwArt,Erwerbsdatum,...,Baureifgest,% Widmung,Baurecht,Bis,auf EZ,Stammeinlage,sonst_wid,sonst_wid_prz,ber. Kaufpreis,Bauzins
_id,,,,,,,,,,,,,,,,,,,,,
6263f0b5eb832bd5a224c7ac,1654,Eßling,4521.0,1220.0,Telephonweg,westl.,447,10000.0,Kaufvertrag,01.01.1990,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,87207.0,NaN
6263f0b5eb832bd5a224e1bc,1010,Neubau,1059.0,1070.0,SCHRANKGASSE,4,185,837.0,Kaufvertrag,01.01.1996,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,4687398.0,NaN
6263f0b5eb832bd5a224e173,1010,Neubau,213.0,1070.0,STIFTGASSE,6,188,862.0,Kaufvertrag,01.01.1996,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,6758574.0,NaN
6263f0b5eb832bd5a2250348,1101,Favoriten,2507.0,1100.0,Quellenstraße,41,1423/31,671.0,Kaufvertrag,01.01.2001,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,NaN,NaN
6263f0b5eb832bd5a2251388,1620,Brigittenau,5909.0,1220.0,Universumstraße,24-26,4680/2,1338.0,Baurechtsvertrag,01.01.2003,...,FALSCH,100.0,WAHR,31.12.2073,5909,FALSCH,NaN,NaN,NaN,"EUR 48.530,44"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263f0b6eb832bd5a22577c7,1107,Simmering,6018.0,NaN,NaN,NaN,NaN,NaN,Kaufvertrag,NaN,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,NaN,NaN
6263f0b6eb832bd5a2257abd,1206,Hütteldorf,2227.0,1140.0,Kometengasse,2,781/248,604.0,Kaufvertrag,NaN,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,NaN,NaN
6263f0b6eb832bd5a2257c05,1008,Margarethen,NaN,NaN,NaN,NaN,NaN,NaN,Kaufvertrag,NaN,...,FALSCH,100.0,FALSCH,NaN,NaN,FALSCH,NaN,NaN,NaN,NaN
